In [1]:
import numpy as np
import re
import random

import tensorflow as tf
import sklearn as sk

import Functions as fn
from DS import DS
from Set import pool
from Iterator import Iterator
from FFModel import FF_Model

# Data Loading

In [2]:
Dataset = pool()
Dataset.load_texts('raw_texts')
Dataset.load_labels('raw_labels')

Raw Text Load Complete
Raw Labels Load Complete


In [3]:
target_dict = fn.load_labels('labels')

Label Load Complete


# Embedding Generation

In [4]:
from gensim.models import Word2Vec

sentences = fn.load_sentences('sentences')

#model = Word2Vec(sentences, min_count=1, size=50)
#model.save('W2V')

model = Word2Vec.load('W2V')
vocab = model.wv.vocab.keys()

C:\Users\Luka\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Sentence Load Complete


In [5]:
#model.most_similar("prozac", topn=10)

# Window Testing

In [7]:
train_cases = Dataset.get_DS(stage = 'test', labelled='yes')
test_cases = Dataset.get_DS(stage = 'train', labelled='yes')
print(len(train_cases.data))
print(len(test_cases.data))

248
10


In [9]:
average_row_size = sum([len(row) for case in labelled_cases.data for row in case.token_text]) / len([1 for case in labelled_cases.data for row in case.token_text])
print(average_row_size)

8.541140071801758


In [11]:
meds = 0
for case in labelled_cases.data:
    meds += len(case.raw_labels.split('\n'))
    
print(meds)

9504


In [10]:
sets = {}

for case in labelled_cases.data:
    temp = cse

[['record', '#<num>'],
 ['<num>',
  '|',
  'tmh',
  '|',
  '<num>',
  '|',
  '|',
  '<num>',
  '|',
  '<num>/<num>/<num>',
  '<num>:<num>:<num>',
  'am',
  '|',
  'discharge',
  'summary',
  '|',
  'signed',
  '|',
  'dis',
  '|',
  'admission',
  'date',
  '<num>/<num>/<num>',
  'report',
  'status',
  'signed'],
 ['discharge', 'date', '<num>/<num>/<num>'],
 ['history',
  'ms',
  'pizzo',
  'is',
  'a',
  '<num>-year-old',
  'woman',
  'with',
  'peripheral',
  'vascular'],
 ['disease', 'who', 'recently', 'underwent', 'revision', 'of', 'her', 'left'],
 ['superior',
  'femoral',
  'artery',
  'anterior',
  'tibial',
  'bypass',
  'graft',
  ',',
  'who',
  'now'],
 ['presents', 'with', 'a', 'cool', ',', 'ischemic', 'left', 'foot'],
 ['mr',
  'denman',
  'is',
  'a',
  '<num>-year-old',
  ',',
  'insulin-dependent',
  'diabetic',
  'with',
  'a',
  'long'],
 ['history',
  'of',
  'peripheral',
  'vascular',
  'disease',
  'as',
  'well',
  'as',
  'multiple',
  'surgical'],
 ['procedure

In [ ]:
print('Ratio: med: {:.2f}% non-med: {:.2f}%'.format(*(np.array(sets['train_labels']).sum(0)/ len(sets['train_labels']))))
fn.saturate_training_set_training(sets, 0.1)
print('Ratio: med: {:.2f}% non-med: {:.2f}%'.format(*(np.array(sets['train_labels']).sum(0)/ len(sets['train_labels']))))

In [ ]:
NN = FF_Model(input_size=1050, layers=[50])
NN.build_graph()
_,_,_,_ = NN.train(sets, epochs=20, batch=30, show_progress=True, show_plot=True)

In [ ]:
res = NN.predict(sets['test_set'])
tru = np.argmax(sets['test_labels'], 1)

In [ ]:
TP = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 0)])
TN = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 1)])
FP = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 1)])
FN = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 0)])
TFPN = [TP, TN, FP, FN]
print('TP\tTN\tFP\tFN\n{}\t{}\t{}\t{}'.format(*TFPN))

In [ ]:
[(sets['test_words'][a], res[a], tru[a]) for a in range(len(sets['test_words'])) if res[a] == 0 and tru[a] == 1]

In [ ]:
proba = []
for i in range(len(res)):
    if not (sets['test_words'][i] == ')' or sets['test_words'][i] == '('):
        proba.append([sets['test_words'][i], res[i], tru[i]])

In [ ]:
sk.metrics.f1_score(np.array(proba)[:,2].astype(int), np.array(proba)[:,1].astype(int), pos_label=0, average='binary')

In [ ]:
len([(proba[a][0], proba[a][1], proba[a][2]) for a in range(len(proba)) if proba[a][1] == 0 and proba[a][2] == 1])

In [ ]:
[(proba[a][0], proba[a][1], proba[a][2]) for a in range(len(proba)) if proba[a][1] == 1 and proba[a][2] == 0]

# HyperParameter Scan

In [ ]:
emb_sizes = [100]
emb_models = 1
target_saturations = [0.05, 0.1, 0.2, 0.5, 0.7]
layer_sizes = [50]
dropouts = [1.0]
learn_rates = [0.01]
epochs = [100]
NN_num = 5

case_num = len(emb_sizes)*emb_models*len(layer_sizes)*len(target_saturations)*len(epochs)*len(dropouts)*len(learn_rates)*NN_num
print(case_num)

In [ ]:
max_performance = 0
n = 1

for emb_size in emb_sizes:
    print('Model Number: %d/%d' %(n, case_num))
    for i in range(emb_models):
        model = Word2Vec(sentences, min_count=1, size=emb_size)
        for saturation in target_saturations:
            sets = fn.get_traintest2 (labelled_cases, model)
            fn.saturate_training_set(sets, model, target_dict['medications'], saturation)
            for layer_size in layer_sizes:
                for drop in dropouts:
                    for rate in learn_rates:
                        for epoch in epochs:
                            for j in range(NN_num):
                                print('Model Number: %d/%d' %(n, case_num))
                                print('ES: %d EM: %d sat: %f, LS: %d, drop: %f, LR: %f, epochs: %d, NN: %d' \
                                       % (emb_size, i, saturation, layer_size, drop, rate, epoch, j))
                                NN = FF_Model(input_size=emb_size, layers=[layer_size], dropout=drop, learn_rate=rate)
                                NN.build_graph()
                                NN.train(sets, epochs=epoch)
                                res = NN.predict(sets['test_set'])
                                tru = np.argmax(sets['test_labels'], 1)
                                perf = sk.metrics.f1_score(tru, res, pos_label=0)
                                if perf > max_performance:
                                    max_performance = perf
                                    NN.save_model('gold')
                                    model.save('gold/GOLDEMB')
                                NN.close()
                                n += 1